In [1]:
import json
import requests
from typing import List, Optional
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_tavily import TavilySearch




In [2]:
# =============================================================================
# 1) Estructuras Pydantic (contratos de salida de tools)
# =============================================================================

class SearchHit(BaseModel):
    title: str = Field(..., description="Título del resultado")
    url: str = Field(..., description="URL del resultado")
    content: Optional[str] = Field(None, description="Extracto o snippet del contenido")


class TavilySearchResult(BaseModel):
    query: str
    hits: List[SearchHit]
    source: str = "tavily"


class WeatherNowResult(BaseModel):
    city_input: str
    city_resolved: str
    temperature_c: float
    time: str
    source: str = "open-meteo"


class ToolError(BaseModel):
    error: str
    source: str




In [3]:
# =============================================================================
# 2) Tools (cada una devuelve JSON con la estructura definida)
# =============================================================================

# Tool A: búsqueda web (Tavily), pero envuelta con contrato Pydantic
tavily_client = TavilySearch(max_results=5)  # requiere TAVILY_API_KEY


@tool
def web_search(query: str) -> str:
    """
    Busca en la web y devuelve un JSON con estructura TavilySearchResult:
    { query, hits: [{title,url,content}], source }
    """
    try:
        raw = tavily_client.invoke(query)

        # raw suele traer "results" como lista de dicts con title/url/content
        results = raw.get("results", []) if isinstance(raw, dict) else []

        hits = []
        for r in results:
            hits.append(
                SearchHit(
                    title=str(r.get("title", "")),
                    url=str(r.get("url", "")),
                    content=r.get("content"),
                )
            )

        structured = TavilySearchResult(query=query, hits=hits)
        return structured.model_dump_json(ensure_ascii=False)

    except Exception as e:
        return ToolError(error=f"web_search failed: {e}", source="tavily").model_dump_json(
            ensure_ascii=False
        )


# Tool B: clima actual (Open-Meteo) con contrato Pydantic
@tool
def get_weather(city: str) -> str:
    """
    Obtiene la temperatura actual (°C) de una ciudad usando Open-Meteo.
    Devuelve un JSON con estructura WeatherNowResult.
    """
    try:
        # 1) Geocoding
        geo = requests.get(
            "https://geocoding-api.open-meteo.com/v1/search",
            params={"name": city, "count": 1, "language": "es", "format": "json"},
            timeout=15,
        ).json()

        if not geo.get("results"):
            return ToolError(error=f"No pude geocodificar '{city}'.", source="open-meteo").model_dump_json(
                ensure_ascii=False
            )

        r0 = geo["results"][0]
        lat, lon = r0["latitude"], r0["longitude"]
        resolved = f'{r0.get("name", city)}, {r0.get("country", "")}'.strip(", ")

        # 2) Current weather
        forecast = requests.get(
            "https://api.open-meteo.com/v1/forecast",
            params={
                "latitude": lat,
                "longitude": lon,
                "current_weather": True,
                "temperature_unit": "celsius",
            },
            timeout=15,
        ).json()

        cw = forecast.get("current_weather")
        if not cw:
            return ToolError(error="No pude obtener current_weather.", source="open-meteo").model_dump_json(
                ensure_ascii=False
            )

        structured = WeatherNowResult(
            city_input=city,
            city_resolved=resolved,
            temperature_c=float(cw.get("temperature")),
            time=str(cw.get("time")),
        )
        return structured.model_dump_json(ensure_ascii=False)

    except Exception as e:
        return ToolError(error=f"get_weather failed: {e}", source="open-meteo").model_dump_json(
            ensure_ascii=False
        )




In [4]:
# =============================================================================
# 3) Agente: decide qué tool usar (o ninguna)
# =============================================================================

def build_agent():
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    tools = [web_search, get_weather]

    system_prompt = (
        "Eres un agente en español con herramientas.\n\n"
        "Tienes dos tools:\n"
        "1) web_search(query): útil para clima PROMEDIO/TÍPICO, estaciones, mejor época, patrones generales.\n"
        "   Devuelve JSON con estructura TavilySearchResult: {query, hits:[{title,url,content}], source}.\n"
        "2) get_weather(city): útil para clima ACTUAL (ahora/en este momento/temperatura actual).\n"
        "   Devuelve JSON con estructura WeatherNowResult o ToolError.\n\n"
        "Reglas de decisión:\n"
        "- Si piden 'promedio', 'típico', 'en general', 'estaciones', 'mejor época' -> usa web_search.\n"
        "- Si piden 'ahora', 'en este momento', 'temperatura actual' -> usa get_weather.\n"
        "- Si la pregunta es conceptual -> no uses tools.\n"
        "- Si falta la ciudad, pregunta por la ciudad antes de usar tools.\n\n"
        "Instrucción clave:\n"
        "Cuando uses tools, lee el JSON devuelto y construye la respuesta final.\n"
        "Si hay error en JSON (ToolError), explícalo y ofrece alternativa."
    )

    return create_agent(
        model=llm,
        tools=tools,
        system_prompt=system_prompt,
        debug=True,  # muestra en consola cuándo llamó web_search o get_weather
    )


def ask(agent, question: str) -> str:
    result = agent.invoke({"messages": [HumanMessage(content=question)]})
    last = result["messages"][-1]
    return getattr(last, "content", last)



In [5]:

agent = build_agent()

print("\n" + "=" * 90)
print("DEMO 1 (promedio -> web_search)")
print(ask(agent, "¿Cómo es en promedio el clima en Medellín durante el año? (temperaturas típicas y lluvias)"))

print("\n" + "=" * 90)
print("DEMO 2 (ahora -> get_weather)")
print(ask(agent, "¿Cuál es la temperatura en este momento en Medellín?"))

print("\n" + "=" * 90)
print("DEMO 3 (ninguna tool)")
print(ask(agent, "Explícame la diferencia entre clima y tiempo atmosférico."))



DEMO 1 (promedio -> web_search)
[values] {'messages': [HumanMessage(content='¿Cómo es en promedio el clima en Medellín durante el año? (temperaturas típicas y lluvias)', additional_kwargs={}, response_metadata={}, id='7624c069-d167-4528-a197-0f308339c91c')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 350, 'total_tokens': 371, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_009ac31db9', 'id': 'chatcmpl-D8mIBF5O26IKIn8LECLuhozYj1H05', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c56e8-e9a1-76a2-ad9b-4aa275929da8-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'clim